# Multi Agent Debate Example

In [1]:
String userHomeDir = System.getProperty("user.home");
String localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";
String langchain4jVersion = "0.35.0";
String dotenvJavaVersion = "3.0.2";

add local maven repository

In [ ]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
%dependency /list-repos


Remove installed package from Jupiter cache

In [3]:
%%bash 
rm -rf \{userHomeDir}/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j

Install required maven dependencies

In [ ]:
%dependency /add org.slf4j:slf4j-jdk14:2.0.9
%dependency /add org.bsc.langgraph4j:langgraph4j-core-jdk8:1.0-20241024
%dependency /add org.bsc.langgraph4j:langgraph4j-langchain4j:1.0-20241024
%dependency /add org.bsc.langgraph4j:langgraph4j-agent-executor:1.0-20241024
%dependency /add dev.langchain4j:langchain4j:\{langchain4jVersion}
%dependency /add dev.langchain4j:langchain4j-open-ai:\{langchain4jVersion}
%dependency /add io.github.cdimascio:dotenv-java:\{dotenvJavaVersion}

%dependency /resolve

In [5]:
import io.github.cdimascio.dotenv.Dotenv;
Dotenv dotenv = Dotenv.load();
String API_KEY = dotenv.get("OPENAI_API_KEY");
String OPENAI_API_KEY = API_KEY;

Initialize Logger

In [6]:
try( var file = new java.io.FileInputStream("./logging.properties")) {
    var lm = java.util.logging.LogManager.getLogManager();
    lm.checkAccess(); 
    lm.readConfiguration( file );
}

var log = org.slf4j.LoggerFactory.getLogger("AgentExecutor");


In [7]:
import dev.langchain4j.service.AiServices;
import dev.langchain4j.service.SystemMessage;
import dev.langchain4j.service.UserMessage;
import dev.langchain4j.service.V;
import dev.langchain4j.model.chat.ChatLanguageModel;
import dev.langchain4j.model.openai.OpenAiChatModel;
import org.bsc.langgraph4j.StateGraph;
import org.bsc.langgraph4j.state.AgentState;
import org.bsc.langgraph4j.state.Channel;
import org.bsc.langgraph4j.state.AppenderChannel;

import java.util.*;
import static org.bsc.langgraph4j.StateGraph.END;
import static org.bsc.langgraph4j.StateGraph.START;
import static org.bsc.langgraph4j.action.AsyncEdgeAction.edge_async;
import static org.bsc.langgraph4j.action.AsyncNodeAction.node_async;


In [8]:
/**
 * ディベートメッセージ
 * speaker: 発言者
 * content: 内容
 * timestamp: タイムスタンプ
 */
 record DebateMessage(String speaker, String content, long timestamp) implements java.io.Serializable {}

In [9]:

class DebateState extends AgentState {
    static Map<String, Channel<?>> SCHEMA = Map.of(
        "debate_history", AppenderChannel.<DebateMessage>of(ArrayList::new),  // 議論履歴
        "current_speaker", Channel.of(() -> ""),                             // 現在の発言者
        "turn_count", Channel.of(() -> 0),                                  // ターン数
        "debate_topic", Channel.of(() -> "")                                // 議論テーマ
    );

    public DebateState(Map<String, Object> initData) {
        super(initData);
    }

    public List<DebateMessage> getHistory() {
        return this.<List<DebateMessage>>value("debate_history").orElseGet(ArrayList::new);
    }
    
    public Optional<String> getCurrentSpeaker() {
        return value("current_speaker");
    }
    
    public int getTurnCount() {
        return this.<Integer>value("turn_count").orElse(0);
    }
    
    public String getDebateTopic() {
        return this.<String>value("debate_topic").orElseThrow();
    }
}


In [10]:
// java.util.concurrent.CompletionException: dev.langchain4j.exception.IllegalConfigurationException: Parameter 'role' of method 'generateArgument' should be annotated with @V or @UserMessage or @UserName or @MemoryId

interface DebateAgent {
    @SystemMessage("あなたは{{topic}}についての議論で{{role}}の立場です。" + 
                  "これまでの議論を考慮して、強力な反論を提供してください。")
    @UserMessage("""
        議題: {{topic}}
        これまでの議論:
        {{history}}
        
        あなたの立場からの次の主張を述べてください。
        以下の点に注意して発言してください：
        - 簡潔で明確な主張を心がけてください
        - 相手の前回の発言に対する反論を含めてください
        - 具体的な例や証拠を示すと良いでしょう
        """)
    String generateArgument(@V("role") String role, @V("topic") String topic, @V("history") String history);
}

In [16]:
interface JudgeAssistant {  
    @UserMessage("""
        以下の議論を分析し、勝者を決定し、その理由を説明してください：
        
        ＜議論内容＞
        {{history}}
        
        以下の点を考慮して判断してください：
        - 論理的な議論展開
        - 証拠や具体例の提示
        - 相手の主張に対する効果的な反論
        - 主張の一貫性
        
        結論は以下の形式で示してください：
        勝者：[賛成派/反審派]
        理由：[詳細な説明]
        """)
    String judgeDebate(@V("history") List<DebateMessage> history);    
}

In [17]:
ChatLanguageModel model = OpenAiChatModel.builder()
            .apiKey(OPENAI_API_KEY)
            .modelName("gpt-4o-mini")
            .logRequests(true)
            .logResponses(true)  
            .temperature(0.7)
            .build();
            
DebateAgent proAgent = AiServices.builder(DebateAgent.class)
            .chatLanguageModel(model)
            .build();

DebateAgent conAgent = AiServices.builder(DebateAgent.class)
            .chatLanguageModel(model)
            .build();

JudgeAssistant judgeAssistant = AiServices.builder(JudgeAssistant.class)
            .chatLanguageModel(model)
            .build();

private final String debateTopic = "AIは規制すべきか？";
private final String proRole = "AI規制推進派";
private final String conRole = "AI自由開発派";

In [18]:
String formatHistory(List<DebateMessage> history) {
        StringBuilder sb = new StringBuilder();
        for (DebateMessage msg : history) {
            sb.append(msg.speaker())
              .append(": ")
              .append(msg.content())
              .append("\n");
        }
        return sb.toString();
    }

In [20]:
StateGraph<DebateState> graph = new StateGraph<>(DebateState.SCHEMA, DebateState::new)
.addEdge(START, "pro_turn")
// 賛成側のターン
.addNode("pro_turn", node_async( state -> {
    try {
    String history = formatHistory(state.getHistory());
    String argument = proAgent.generateArgument(proRole, debateTopic, history);        
    DebateMessage message = new DebateMessage(proRole, argument, System.currentTimeMillis());
    log.info( message.toString() );
    return Map.of(
        "debate_history", message,
        "current_speaker", conRole,
        "turn_count", state.getTurnCount() + 1
    );
    } catch( Exception e ) {
        e.printStackTrace();
        return Map.of();
    }
}))
// 反対側のターン
.addNode("con_turn", node_async(state -> {
    try {
        String history = formatHistory(state.getHistory());
        String argument = conAgent.generateArgument(conRole, debateTopic, history);        
        DebateMessage message = new DebateMessage(conRole, argument, System.currentTimeMillis());
        log.info( message.toString() );
        return Map.of(
            "debate_history", message,
            "current_speaker", proRole,
            "turn_count", state.getTurnCount() + 1
        );
    } catch( Exception e ) {
        e.printStackTrace();
        return Map.of();
    }
}
    ))
    // 判定ノード
    .addNode("judge", node_async(state -> {
        String result = judgeAssistant.judgeDebate(state.getHistory());
        return Map.of("debate_result", result);
    }))
    // 条件付きエッジの設定
    .addConditionalEdges(
        "pro_turn",
        edge_async(state -> state.getTurnCount() >= 6 ? "judge" : "con_turn"),
        Map.of("con_turn", "con_turn", "judge", "judge")
    )
    .addConditionalEdges(
        "con_turn", 
        edge_async(state -> state.getTurnCount() >= 6 ? "judge" : "pro_turn"),
        Map.of("pro_turn", "pro_turn", "judge", "judge")
    )
    .addEdge("judge", END)

In [ ]:
Map<String, Object> initialState = new HashMap<>();
initialState.put("turn_count", 0);
initialState.put("current_speaker", "pro");
initialState.put("debate_history", new ArrayList<DebateMessage>());
        
var workflow = graph.compile();
workflow.invoke(initialState);